# **LLM-BASED MEDIA RECOMMENDER SYSTEM**

## **Summary**

This notebook presents a study exploring the use of Large Language Models (LLMs) as zero-shot recommendation engines across three media domains: anime, books, and movies.

We use publicly available datasets from Kaggle containing metadata and synopses. The workflow for each domain is as follows:

1. The user provides a natural language prompt describing their current mood or preferences.
2. We embed both the prompt and all item descriptions using `all-MiniLM-L6-v2` from the SentenceTransformers library.
3. Cosine similarity is computed between the user input and each item’s synopsis to shortlist the top 15 most relevant titles.
4. These 15 items are passed as context to `gpt-3.5-turbo`, which selects and justifies one final recommendation.
5. We evaluate the results using two cosine similarity scores:
   - Between the user prompt and the recommended **synopsis**.
   - Between the user prompt and the GPT-generated **justification/reasoning**.

This project aims to assess how well LLMs can perform personalized media recommendations without explicit training, using only natural language inputs and existing content metadata.

## **Datasets**

- Animes: https://www.kaggle.com/datasets/tanishksharma9905/top-popular-anime
- Books: https://www.kaggle.com/datasets/abdallahwagih/books-dataset
- Movies: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset



## **Data Formatting**

In [20]:
import pandas as pd

anime_df = pd.read_csv("popular_anime.csv")

# Check total entries and columns
print("Total anime entries:", len(anime_df))
print("Columns:", anime_df.columns)

# Select and rename relevant columns
anime_df = anime_df[['name', 'genres', 'synopsis', 'score']].dropna()
anime_df.columns = ['Title', 'Genres', 'Synopsis', 'Score']

# Sort by score 
anime_df = anime_df.sort_values(by='Score', ascending=False)

# Format as list of strings for LLM input
anime_data = []
for _, row in anime_df.iterrows():
    entry = f"""Title: {row['Title']}
Genres: {row['Genres']}
Score: {row['Score']}
Synopsis: {row['Synopsis']}"""
    anime_data.append(entry)

# sample
print("\nSample formatted anime entry:\n")
print(anime_data[0])


Total anime entries: 28825
Columns: Index(['id', 'name', 'genres', 'type', 'episodes', 'status', 'aired_from',
       'aired_to', 'duration_per_ep', 'score', 'scored_by', 'rank', 'rating',
       'studios', 'producers', 'image', 'trailer', 'synopsis'],
      dtype='object')

Sample formatted anime entry:

Title: Frieren: Beyond Journey's End
Genres: Adventure, Drama, Fantasy
Score: 9.3
Synopsis: During their decade-long quest to defeat the Demon King, the members of the hero's party—Himmel himself, the priest Heiter, the dwarf warrior Eisen, and the elven mage Frieren—forge bonds through adventures and battles, creating unforgettable precious memories for most of them.

However, the time that Frieren spends with her comrades is equivalent to merely a fraction of her life, which has lasted over a thousand years. When the party disbands after their victory, Frieren casually returns to her "usual" routine of collecting spells across the continent. Due to her different sense of time, she s

In [67]:
books_df = pd.read_csv("books_data.csv")

# Inspect basic info
print("Total books entries:", len(books_df))
print("Columns:", books_df.columns)

# Keep only the relevant columns
books_df = books_df[['title', 'authors', 'description', 'average_rating', 'categories']].dropna()
books_df.columns = ['Title', 'Author', 'Description', 'Rating', 'Genres']

# Sort by rating
books_df = books_df.sort_values(by='Rating', ascending=False)

# Format as list of strings for LLM input
books_data = []
for _, row in books_df.iterrows():
    entry = f"""Title: {row['Title']}
Author: {row['Author']}
Rating: {row['Rating']}
Genres: {row['Genres']}
Description: {row['Description']}"""
    books_data.append(entry)

# sample
print("\nSample formatted book entry:\n")
print(books_data[0])


Total books entries: 6810
Columns: Index(['isbn13', 'isbn10', 'title', 'subtitle', 'authors', 'categories',
       'thumbnail', 'description', 'published_year', 'average_rating',
       'num_pages', 'ratings_count'],
      dtype='object')

Sample formatted book entry:

Title: Bill Gates
Author: Sara Barton-Wood
Rating: 5.0
Genres: Juvenile Nonfiction
Description: Presents the life of Bill Gates, from his childhood, to his education, and to his days as mastermind and head of Microsoft a multi-billion dollar computer software company.


In [24]:
import ast
  
movies_df = pd.read_csv("movies_metadata.csv", low_memory=False)

# Show basic info
print("Total movie entries:", len(movies_df))
print("Columns:", movies_df.columns)

# Filter out bad rows
movies_df = movies_df[['title', 'overview', 'vote_average', 'genres']].dropna()
movies_df = movies_df[movies_df['vote_average'] > 0]

# Process genres column: convert stringified list of dicts to comma-separated string
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return ", ".join([g['name'] for g in genres])
    except:
        return ""

movies_df['genres'] = movies_df['genres'].apply(extract_genres)

# Rename for consistency
movies_df.columns = ['Title', 'Description', 'Rating', 'Genres']

# Sort by rating
movies_df = movies_df.sort_values(by='Rating', ascending=False)

# Format into LLM-friendly prompt entries
movies_data = []
for _, row in movies_df.iterrows():
    entry = f"""Title: {row['Title']}
Genres: {row['Genres']}
Rating: {row['Rating']}
Description: {row['Description']}"""
    movies_data.append(entry)

# sample
print("\nSample formatted movie entry:\n")
print(movies_data[0])


Total movie entries: 45466
Columns: Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Sample formatted movie entry:

Title: Pourquoi Israël
Genres: Documentary
Rating: 10.0
Description: Using interviews and other footage shot especially for this documentary, French director Claude Lanzmann investigates the state of Israel in 1972. This movie concentrates on Israelis going about their business of everyday living.


In [26]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41710 entries, 32949 to 25444
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        41710 non-null  object 
 1   Description  41710 non-null  object 
 2   Rating       41710 non-null  float64
 3   Genres       41710 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.6+ MB


## **Feasibility: testing on a small sample**

In [28]:
pip install openai


Note: you may need to restart the kernel to use updated packages.


In [39]:
import openai
import random

# Set API key
openai.api_key = "My_API_Key" #Removed the actual key for security purposes

#  Prepare the top anime dataset (limit to top 50 based on score)
top_anime_df = anime_df.sort_values(by="Score", ascending=False).dropna().head(50)

# Create a function to turn the dataframe into a prompt string
def build_anime_context(df):
    context = ""
    for idx, row in df.iterrows():
        entry = f"Title: {row['Title']}\nGenres: {row['Genres']}\nSynopsis: {row['Synopsis']}\nScore: {row['Score']}\n"
        context += entry + "\n---\n"
    return context

anime_context = build_anime_context(top_anime_df)

# Function to ask LLM to recommend
def recommend_anime(user_prompt, context):
    system_msg = "You are a recommendation system that suggests anime based on the user's preferences using the provided list."
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"Here is a list of anime:\n\n{context}"},
        {"role": "user", "content": f"My preferences: {user_prompt}\nBased on the above list, which anime should I watch and why?"}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )

    return response['choices'][0]['message']['content']

# usage
user_input = "I'm in the mood for something dramatic and psychological with a strong female lead"
response = recommend_anime(user_input, anime_context)
print("\n Recommended Anime:\n")
print(response)



🎯 Recommended Anime:

Based on your preferences for a dramatic and psychological anime with a strong female lead, I recommend **"A Silent Voice"**. 

**Title: A Silent Voice**  
**Genres:** Drama, Award Winning  
**Synopsis:** Shouya Ishida, a former elementary school student, seeks redemption after bullying a deaf classmate named Shouko Nishimiya. Years later, Shouya is plagued by his past actions and sets out to make amends by reconnecting with Shouko. The anime explores themes of guilt, forgiveness, redemption, and the complexities of human relationships. The strong female lead, Shouko, showcases resilience and compassion in the face of adversity, making her a compelling character to follow.

**Why you should watch it:**  
1. **Emotional Depth:** "A Silent Voice" delves into heavy themes like bullying, mental health, and redemption, offering a poignant and emotional viewing experience.
2. **Character Development:** The characters undergo significant growth and introspection through

## **Production / Prototype**

In [45]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 66.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### **Animes**

In [179]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import openai
import numpy as np

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# USER PROMPT
user_prompt = "I'm in the mood for something dramatic and psychological with a strong female lead"

# Embed all anime synopses
synopses = anime_df['Synopsis'].fillna("").tolist()
synopsis_embeddings = embedder.encode(synopses, convert_to_tensor=True)

# Embed the user input
user_embedding = embedder.encode(user_prompt, convert_to_tensor=True)

# Cosine similarity
cos_similarities = cosine_similarity(
    user_embedding.cpu().numpy().reshape(1, -1),
    synopsis_embeddings.cpu().numpy()
)[0]

# Get top 15 most relevant anime
top_indices = np.argsort(cos_similarities)[::-1][:15]
top_anime_df = anime_df.iloc[top_indices]


# Format the context for GPT
anime_context = "\n\n".join(
    f"Title: {row['Title']}\nGenres: {row['Genres']}\nSynopsis: {row['Synopsis']}"
    for _, row in top_anime_df.iterrows()
)

# Ask GPT for a recommendation from the shortlist
def recommend_anime(user_prompt, context):
    system_msg = "You are a recommendation system that selects the best anime based on user preferences and explains your choice. Only use the provided list."
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"Here is a list of anime:\n\n{context}"},
        {"role": "user", "content": f"My preferences: {user_prompt}\nWhich anime should I watch and why? Provide the title, genres, synopsis, and your reasoning."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )

    return response['choices'][0]['message']['content']

# Get recommendation
response = recommend_anime(user_prompt, anime_context)
print(response)


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


I recommend you watch "Sentimental Journey".

Title: Sentimental Journey
Genres: Drama, Romance
Synopsis: Sentimental Journey is a collection of twelve short stories about twelve different girls. Though unrelated, there is one common theme that binds the episodes together: each girl's bittersweet experience of f

Reasoning:
"Sentimental Journey" fits your preference for something dramatic and psychological with a strong female lead. Each story in this anime focuses on a different girl and her emotional journey, allowing for a deep exploration of complex feelings and experiences. The series delves into themes of love, loss, and personal growth, offering a poignant and reflective look at the lives of the female characters. The strong emotional depth and psychological introspection make "Sentimental Journey" a compelling choice for viewers seeking a dramatic and thought-provoking anime with a focus on female protagonists.


In [181]:
import re
from sklearn.metrics.pairwise import cosine_similarity

# Extract Synopsis
synopsis_match = re.search(r"Synopsis:\s*(.+?)\n(?:Justification|Reasoning|$)", response, re.DOTALL)
if synopsis_match:
    recommended_synopsis = synopsis_match.group(1).strip()
else:
    recommended_synopsis = None

#  Extract Justification
justification_match = re.search(r"(Justification|Reasoning):\s*(.+)", response, re.DOTALL)
if justification_match:
    recommended_justification = justification_match.group(2).strip()
else:
    recommended_justification = None

# Compute Similarity Scores
if recommended_synopsis:
    synopsis_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_synopsis])
    )[0][0]
    print(f" Synopsis Similarity Score: {synopsis_score:.3f}")
else:
    print(" Could not extract synopsis.")

if recommended_justification:
    justification_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_justification])
    )[0][0]
    print(f" Justification Similarity Score: {justification_score:.3f}")
else:
    print(" Could not extract justification.")


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 Synopsis Similarity Score: 0.415
 Justification Similarity Score: 0.521


In [191]:
user_prompt = "I'm in the mood for something chaotic"

def recommend_anime(user_prompt, context):
    system_msg = "You are a recommendation system that selects the best anime based on user preferences and explains your choice. Only use the provided list."
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"Here is a list of anime:\n\n{context}"},
        {"role": "user", "content": f"My preferences: {user_prompt}\nWhich anime should I watch and why? Provide the title, genres, synopsis, and your reasoning."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )

    return response['choices'][0]['message']['content']

# Get recommendation
response = recommend_anime(user_prompt, anime_context)
print(response)

Title: Colorful

Genres: Comedy, Ecchi

Synopsis: Colorful! contains over 50 vignettes of men, women, and the pursuit of various comedic and risqué situations. From unexpected encounters on subways to panty-obsessed video geeks, this anime offers a strange, deranged, racy, and hilarious dose of animated chaos. It explores a wide range of scenarios involving quirky characters and bizarre situations, providing a mix of humor and ecchi elements.

Reasoning: If you're in the mood for something chaotic, Colorful is a perfect choice due to its diverse and eccentric collection of short stories. The anime presents a wide array of characters and situations that are bound to keep you entertained and surprised with its unconventional and humorous take on everyday occurrences. The blend of comedy and ecchi elements adds an extra layer of unpredictability and fun to the chaotic nature of the anime, making it an enjoyable watch for those seeking an animated rollercoaster of absurdity and laughter.


In [193]:
synopsis_match = re.search(r"Synopsis:\s*(.+?)\n(?:Justification|Reasoning|$)", response, re.DOTALL)
if synopsis_match:
    recommended_synopsis = synopsis_match.group(1).strip()
else:
    recommended_synopsis = None


justification_match = re.search(r"(Justification|Reasoning):\s*(.+)", response, re.DOTALL)
if justification_match:
    recommended_justification = justification_match.group(2).strip()
else:
    recommended_justification = None

if recommended_synopsis:
    synopsis_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_synopsis])
    )[0][0]
    print(f" Synopsis Similarity Score: {synopsis_score:.3f}")
else:
    print(" Could not extract synopsis.")

if recommended_justification:
    justification_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_justification])
    )[0][0]
    print(f" Justification Similarity Score: {justification_score:.3f}")
else:
    print(" Could not extract justification.")


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 Synopsis Similarity Score: 0.376
 Justification Similarity Score: 0.520


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### **Why Cosine Similarity?** : 
Cosine similarity is a metric that measures how similar two vectors are, based on the angle between them. It’s widely used in natural language processing to compare text embeddings, like comparing a mood prompt to a book or movie synopsis.


### How It Works:

1. **Text is converted into vectors**
   Using a model like `SentenceTransformer`, the input (e.g., "I want something dark and emotional") and a synopsis are transformed into high-dimensional vectors (embeddings). These vectors capture the semantic meaning of the text.

2. **Cosine similarity measures the angle**

   * A cosine of **1.0** means the vectors are pointing in the same direction → **perfect match**.
   * A cosine of **0.0** means the vectors are orthogonal → **no similarity**.
   * A cosine of **-1.0** means they're opposite → **completely different**.

3. **Formula**

   $$
   \text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
   $$

   Where:

   * $A \cdot B$ is the **dot product** of the vectors
   * $\|A\|$ and $\|B\|$ are their magnitudes (lengths)

### Why it’s useful for the project:

* It helps **quantify** how well the LLM’s recommended synopsis matches the intent.
* It provides an **objective metric** to compare different recommendation methods.
* It can be used to **evaluate accuracy** even when there’s no labeled ground truth.


In this case, a **higher cosine similarity score** (closer to 1) means the anime/book/movie recommendation is semantically close to what the user said they wanted, even if the words used are different.


### **Books**

In [165]:
# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# USER PROMPT
user_prompt = "I want to read something mysterious and thought-provoking with deep character development"

# Embed all book synopses
synopses = books_df['Description'].fillna("").tolist()
synopsis_embeddings = embedder.encode(synopses, convert_to_tensor=True)

# Embed the user input
user_embedding = embedder.encode(user_prompt, convert_to_tensor=True)

# Cosine similarity
cos_similarities = cosine_similarity(
    user_embedding.cpu().numpy().reshape(1, -1),
    synopsis_embeddings.cpu().numpy()
)[0]

# Get top 15 most relevant books
top_indices = np.argsort(cos_similarities)[::-1][:15]
top_books_df = books_df.iloc[top_indices]

# Format the context for GPT
book_context = "\n\n".join(
    f"Title: {row['Title']}\nGenres: {row['Genres']}\nSynopsis: {row['Description']}"
    for _, row in top_books_df.iterrows()
)

# Ask GPT for a recommendation
def recommend_book(user_prompt, context):
    system_msg = "You are a book recommendation system that selects the best book based on user preferences and explains your choice. Only use the provided list."
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"Here is a list of books:\n\n{context}"},
        {"role": "user", "content": f"My preferences: {user_prompt}\nWhich book should I read and why? Provide the title, genres, synopsis, and your reasoning."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )

    return response['choices'][0]['message']['content']

# Get recommendation
response = recommend_book(user_prompt, book_context)
print(response)


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Title: The Shadow of the Wind
Genres: Fiction
Synopsis: The international literary sensation--a runaway bestseller in Spain--is about a boy's quest through the secrets and shadows of postwar Barcelona for a mysterious author whose book has proved as dangerous to own as it is impossible to forget.
Reasoning: "The Shadow of the Wind" is the perfect choice for you as it combines mystery, intrigue, and deep character development. The novel is set in a postwar Barcelona filled with secrets and shadows, following a boy's quest to uncover the mysteries surrounding a mysterious author and his dangerous book. The story delves into complex characters, their hidden pasts, and the interconnectedness of their lives, offering a thought-provoking and immersive reading experience. The richly developed characters, along with the atmospheric setting and intricate plot, make this book a compelling and engrossing read that will satisfy your desire for mystery and deep character exploration.


In [169]:
synopsis_match = re.search(r"Synopsis:\s*(.+?)\n(?:Justification|Reasoning|$)", response, re.DOTALL)
if synopsis_match:
    recommended_synopsis = synopsis_match.group(1).strip()
else:
    recommended_synopsis = None


justification_match = re.search(r"(Justification|Reasoning):\s*(.+)", response, re.DOTALL)
if justification_match:
    recommended_justification = justification_match.group(2).strip()
else:
    recommended_justification = None


if recommended_synopsis:
    synopsis_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_synopsis])
    )[0][0]
    print(f"\n Synopsis Similarity Score: {synopsis_score:.3f}")
else:
    print("\n Could not extract synopsis for similarity calculation.")

# --- Compute cosine similarity for justification
if recommended_justification:
    justification_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_justification])
    )[0][0]
    print(f" Justification Similarity Score: {justification_score:.3f}")
else:
    print("\n Could not extract justification for similarity calculation.")



 Synopsis Similarity Score: 0.503
 Justification Similarity Score: 0.598


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### **Movies**

In [171]:
# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# USER PROMPT
user_prompt = "I'm in the mood for a suspenseful thriller with emotional depth and a dark atmosphere"

# Embed all movie overviews
overviews = movies_df['Description'].fillna("").tolist()
overview_embeddings = embedder.encode(overviews, convert_to_tensor=True)

# Embed user input
user_embedding = embedder.encode(user_prompt, convert_to_tensor=True)

# Cosine similarity
cos_similarities = cosine_similarity(
    user_embedding.cpu().numpy().reshape(1, -1),
    overview_embeddings.cpu().numpy()
)[0]

# Get top 15 relevant movies
top_indices = np.argsort(cos_similarities)[::-1][:15]
top_movies_df = movies_df.iloc[top_indices]

# Format GPT context
movie_context = "\n\n".join(
    f"Title: {row['Title']}\nGenres: {row['Genres']}\nOverview: {row['Description']}"
    for _, row in top_movies_df.iterrows()
)

# Ask GPT to recommend one
def recommend_movie(user_prompt, context):
    system_msg = "You are a movie recommendation system that selects one movie from the list and explains why it's a good match. Only use the provided list."
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"Here is a list of movies:\n\n{context}"},
        {"role": "user", "content": f"My preferences: {user_prompt}\nWhich movie should I watch and why? Return the title, genres, overview, and justification."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )

    return response['choices'][0]['message']['content']

# recommendation
response = recommend_movie(user_prompt, movie_context)
print(response)


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Title: The Dark Below
Genres: Thriller
Overview: The Dark Below is an experimental thriller set on Michigan's Great Lakes.

Justification: "The Dark Below" fits your preferences as it is a suspenseful thriller that offers a dark atmosphere and emotional depth. The experimental nature of the film adds an intriguing layer to the storytelling, making it a unique and captivating watch for those looking for a gripping and atmospheric thriller experience.


In [177]:
synopsis_match = re.search(r"Overview:\s*(.+?)\n(?:Justification|Reasoning|$)", response, re.DOTALL)
if synopsis_match:
    recommended_synopsis = synopsis_match.group(1).strip()
else:
    recommended_synopsis = None


justification_match = re.search(r"(Justification|Reasoning):\s*(.+)", response, re.DOTALL)
if justification_match:
    recommended_justification = justification_match.group(2).strip()
else:
    recommended_justification = None


if recommended_synopsis:
    synopsis_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_synopsis])
    )[0][0]
    print(f"\n Synopsis Similarity Score: {synopsis_score:.3f}")
else:
    print("\n Could not extract synopsis for similarity calculation.")

#  Compute cosine similarity for justification
if recommended_justification:
    justification_score = cosine_similarity(
        embedder.encode([user_prompt]),
        embedder.encode([recommended_justification])
    )[0][0]
    print(f" Justification Similarity Score: {justification_score:.3f}")
else:
    print("\n Could not extract justification for similarity calculation.")



 Synopsis Similarity Score: 0.515
 Justification Similarity Score: 0.707


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
